# Projeto Estágio supervisionado - EDA

O conjunto de dados consiste em bases de dados que mostram sobre sua produção(1997 - 2025), processamento(1990 - 2025), importação(2000 - 2025), exportação(2000 - 2025) e, por ultimo, sobre o combustivel automotivo que tem total relação com o petróleo(2004, 2014, 2019, 2020, 2022 e 2025).

O objetivo desde notbook é fazer uma análise dos dados completa para podermos seguir para os próximos passos são envolvendo Modelagem de dados, SQL, POWER BI (Dashboards) e Machine learning.

### Pipeline do Projeto de Análise de Dados de Petróleo e Combustíveis

#### Fase 1: Estruturação e Entendimento (Onde estamos)
1.1. Definição do Problema de Negócio
1.2. Coleta e Análise Exploratória PRELIMINAR
- Para cada arquivo CSV, faremos uma análise inicial (.info(), .head())
- Documentaremos os achados (tipos de dados a corrigir, colunas importantes)
1.3. Modelagem de Dados
- Com base nos achados, vamos desenhar nosso Star Schema (Tabelas Fato e Dimensão)

#### Fase 2: Engenharia de Dados (ETL - Extract, Transform, Load)
2.1. Criação da Estrutura no Banco de Dados
- Escreveremos o código SQL para criar as tabelas vazias no MySQL.
2.2. Desenvolvimento do Script de Carga e Tratamento
- Criaremos o script Python que lê os CSVs, limpa os dados (corrige tipos, trata nulos), e carrega nas tabelas do MySQL.

#### Fase 3: Visualização inicial com POWER BI
- 3.1. Conexão do Power BI ao Banco de Dados
- 3.2. Criação de dashboards iniciais 

#### Fase 4: Análise Exploratória de Dados PROFUNDA (EDA)
- 4.1. Análise Univariada (estudo de cada variável)
- 4.2. Análise Bivariada e Multivariada (cruzamento de variáveis)
- 4.3. Geração de Gráficos e Descoberta de Insights

#### Fase 5: Modelagem Preditiva (Machine Learning)
- 5.1. Preparação dos Dados para o Modelo
- 5.2. Treinamento de Modelos Não-Supervisionados (clusters, anomalias)
- 5.3. Interpretação dos Resultados

#### Fase 6: Visualização e Apresentação (Power BI)
- 6.2. Criação de Dashboards Interativos
- 6.3. Apresentação dos Resultados e Insights



## *Análise Inicial - Produção petroleo*

In [1]:
# Importando bibliotecas
import pandas as pd
import os

In [2]:
# Definindo o caminho do arquivo
pasta_dados = 'Data'
nome_arquivo = 'producao-petroleo-m3-1997-2025.csv'

caminho_completo = os.path.join(pasta_dados, nome_arquivo)

In [4]:
# Carregando o arquivo
try:
    df = pd.read_csv(caminho_completo, sep=';', encoding='utf-8-sig')
    print(f"Arquivo '{nome_arquivo}' carregado com sucesso!")
except FileNotFoundError:
    print(f"ERRO: O arquivo '{caminho_completo}' não foi encontrado.")
except Exception as erro:
    print(f"Ocorreu um erro inesperado: {erro}")

Arquivo 'producao-petroleo-m3-1997-2025.csv' carregado com sucesso!


In [ ]:
# Visualizando o DataFrame de forma inicial
display(df.head())
display(df.tail())
display(df.info(5))


### Análise Preliminar - `producao-petroleo-m3-1997-2025.csv`

#### Estrutura Geral
* O arquivo contém 7.656 registros e 7 colunas.
* A contagem de `non-null` indica que não há valores nulos (faltando) no conjunto de dados.

#### Colunas de Dimensão (Contexto)
* **`ANO`**: Lido corretamente como número (**`int64`**). Será usado na nossa dimensão de tempo.
* **`MÊS`**: Lido como texto (**`object`**). **Ação Necessária:** Converter para um formato numérico (1 para JAN, 2 para FEV, etc.) durante o ETL.
* **`GRANDE REGIÃO`**, **`UNIDADE DA FEDERAÇÃO`**: Lido como texto (**`object`**), o que é esperado. Serão colunas da nossa dimensão de localização.
* **`PRODUTO`**: Lido como texto (**`object`**). Será a base da nossa dimensão de produto.
* **`LOCALIZAÇÃO`**: Lido como texto (**`object`**, com valores 'TERRA' e 'MAR'). Pode se tornar uma coluna na dimensão de produto ou local.

#### Coluna Fato (A Métrica)
* **`PRODUÇÃO`**: Lido como texto (**`object`**). **Ação Crítica Necessária:** Principal métrica do arquivo que precisa ser convertida para um tipo numérico. A causa precisa ser investigada na Fase 2 (ETL).

#### Padronização
* Todos os nomes de colunas precisam ser convertidos para o formato `snake_case` (minúsculas, sem acentos, com `_`).

## *Análise Inicial - Processamento petroleo*

In [7]:
# Definindo o caminho do arquivo
pasta_dados = 'Data'
nome_arquivo = 'processamento-petroleo-m3-1990-2025.csv'

caminho_completo = os.path.join(pasta_dados, nome_arquivo)

In [8]:
# Carregando o arquivo
try:
    df = pd.read_csv(caminho_completo, sep=';', encoding='utf-8-sig')
    print(f"Arquivo '{nome_arquivo}' carregado com sucesso!")
except FileNotFoundError:
    print(f"ERRO: O arquivo '{caminho_completo}' não foi encontrado.")
except Exception as erro:
    print(f"Ocorreu um erro inesperado: {erro}")

Arquivo 'processamento-petroleo-m3-1990-2025.csv' carregado com sucesso!


In [ ]:
# Visualizando o DataFrame de forma inicial
display(df.head())
display(df.tail())
display(df.info(5))

### Análise Preliminar - `processamento-petroleo.csv`

#### Estrutura Geral
* O arquivo possui 22.671 registros e 6 colunas.
* Pela contagem de `non-null`, não há valores nulos aparentes.

#### Colunas de Dimensão (Contexto)
* **`ANO`** e **`MÊS`**: Padrão já identificado. `ANO` está correto como `int64`, e `MÊS` é `object` (texto), precisando de tratamento futuro.
* **`UNIDADE DA FEDERAÇÃO`**: Coluna em comum com o arquivo de produção, servindo como ponto de conexão.
* **`REFINARIA`**: **Nova Pista!** Forte candidata a se tornar uma nova Tabela Dimensão (`d_refinaria`).
* **`MATÉRIA PRIMA`**: **Nova Pista!** Candidata a uma Tabela Dimensão ou a fazer parte da dimensão de produto (`d_produto`).

#### Coluna Fato (A Métrica)
* **`PROCESSADO`**: **Ótima Notícia!** Diferente do arquivo anterior, esta métrica já foi lida como um número (**`int64`**), indicando dados mais limpos.

#### Padronização
* Assim como no outro arquivo, todos os nomes de colunas precisarão ser padronizados para o formato `snake_case`.

## *Análise Inicial - importações e exportações petroleo*

In [ ]:
# Definindo o caminho do arquivo
pasta_dados = 'Data'
nome_arquivo = 'importacoes-exportacoes-petroleo-2000-2025.csv'

caminho_completo = os.path.join(pasta_dados, nome_arquivo)

In [ ]:
# Carregando o arquivo
try:
    df = pd.read_csv(caminho_completo, sep=';', encoding='utf-8-sig')
    print(f"Arquivo '{nome_arquivo}' carregado com sucesso!")
except FileNotFoundError:
    print(f"ERRO: O arquivo '{caminho_completo}' não foi encontrado.")
except Exception as erro:
    print(f"Ocorreu um erro inesperado: {erro}")

In [ ]:
# Visualizando o DataFrame de forma inicial
display(df.head())
display(df.tail())
display(df.info(5))

### Análise Preliminar - `importacoes-exportacoes-petroleo.csv`

#### Estrutura Geral
* O arquivo é "leve", contendo 614 registros e 6 colunas, ideal para a nossa análise estrutural.
* A contagem de `non-null` indica que não há valores nulos aparentes.

#### Colunas de Dimensão (Contexto)
* **`ANO`**, **`MÊS`**, **`PRODUTO`**: Dimensões que se repetem em outros arquivos, confirmando que serão nossas chaves de conexão principais. `MÊS` continua sendo `object` e precisará de tratamento.
* **`OPERAÇÃO COMERCIAL`**: **Pista Importantíssima!** Esta é uma nova dimensão categórica, com valores "IMPORTAÇÃO" e "EXPORTAÇÃO". Ela será fundamental para separar nossas métricas.

#### Colunas Fato (As Métricas)
* Este arquivo é rico em métricas, contendo duas colunas de fatos.
* **`IMPORTADO / EXPORTADO`**: Representa o volume/peso da operação. **Ação Crítica Necessária:** Foi lido como texto (**`object`**) mas claramente contém números. Precisa ser convertido para um tipo numérico (float/decimal) na fase de ETL.
* **`DISPÊNDIO / RECEITA`**: Representa o valor monetário da operação. **Boa Notícia:** Foi lido corretamente como um número (**`int64`**), indicando dados mais limpos nesta coluna.

#### Padronização
* Os nomes das colunas (ex: `IMPORTADO / EXPORTADO`) contêm espaços e `/`. Precisarão ser padronizados para `snake_case` (ex: `volume_kg`, `valor_usd`).

## *Análise Inicial - importações e exportações derivados*

In [ ]:
# Definindo o caminho do arquivo
pasta_dados = 'Data'
nome_arquivo = 'importacoes-exportacoes-derivados-2000-2025.csv'

caminho_completo = os.path.join(pasta_dados, nome_arquivo)

In [ ]:
# Carregando o arquivo
try:
    df = pd.read_csv(caminho_completo, sep=';', encoding='utf-8-sig')
    print(f"Arquivo '{nome_arquivo}' carregado com sucesso!")
except FileNotFoundError:
    print(f"ERRO: O arquivo '{caminho_completo}' não foi encontrado.")
except Exception as erro:
    print(f"Ocorreu um erro inesperado: {erro}")

In [ ]:
# Visualizando o DataFrame de forma inicial
display(df.head())
display(df.tail())
display(df.info(5))

### Análise Preliminar - `importacoes-exportacoes-derivados.csv`

#### Estrutura Geral
* O arquivo contém 9.192 registros e 6 colunas.
* A contagem de `non-null` indica, novamente, que não há valores nulos aparentes.

#### Colunas de Dimensão (Contexto)
* **Estrutura Idêntica:** As colunas de dimensão (`ANO`, `MÊS`, `PRODUTO`, `OPERAÇÃO COMERCIAL`) são **exatamente as mesmas** do arquivo de importação/exportação de petróleo cru. Isso é ótimo, pois significa que poderemos usar a mesma lógica de ETL para tratar ambos os arquivos.
* **`PRODUTO`**: Esta coluna irá enriquecer nossa dimensão de produto com derivados importantes, como 'ASFALTO' e 'ÓLEO DIESEL'.
* **`MÊS`**: Continua sendo `object`, confirmando a necessidade de tratamento.

#### Colunas Fato (As Métricas)
* **Padrão Confirmado:** As duas colunas de métricas seguem o mesmo padrão do arquivo anterior.
* **`IMPORTADO / EXPORTADO`**: **Ação Crítica Necessária:** Confirmado que esta coluna é lida como texto (**`object`**) e precisará ser convertida para um tipo numérico (float/decimal).
* **`DISPÊNDIO / RECEITA`**: Lido corretamente como número (**`int64`**). **Ação de Modelagem:** No Star Schema, esta coluna deverá ser modelada como `DECIMAL(18, 4)` para garantir a precisão financeira.

#### Padronização
* A padronização dos nomes das colunas para o formato `snake_case` será necessária, assim como nos outros arquivos.

## *Análise Inicial - combustivel automotivo 2025.01*

In [5]:
# Definindo o caminho do arquivo
pasta_dados = 'Data'
nome_arquivo = 'ca-2025-01.csv'

caminho_completo = os.path.join(pasta_dados, nome_arquivo)

In [6]:
# Carregando o arquivo
try:
    df = pd.read_csv(caminho_completo, sep=';', encoding='utf-8-sig')
    print(f"Arquivo '{nome_arquivo}' carregado com sucesso!")
except FileNotFoundError:
    print(f"ERRO: O arquivo '{caminho_completo}' não foi encontrado.")
except Exception as erro:
    print(f"Ocorreu um erro inesperado: {erro}")

Arquivo 'ca-2025-01.csv' carregado com sucesso!


C:\Users\User\AppData\Local\Temp\ipykernel_19684\3910595749.py:3: DtypeWarning: Columns (0,1,2,3,4,5,6,7,8,9,10,11,12,14,15) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(caminho_completo, sep=';', encoding='utf-8-sig')


In [7]:
# Visualizando o DataFrame de forma inicial
display(df.head())
display(df.tail())
display(df.info(5))

,Regiao - Sigla,Estado - Sigla,Municipio,Revenda,CNPJ da Revenda,Nome da Rua,Numero Rua,Complemento,Bairro,Cep,Produto,Data da Coleta,Valor de Venda,Valor de Compra,Unidade de Medida,Bandeira
0,NE,CE,SOBRAL,ECONOGÁS DO BRASIL DIST. DERIV. DE PET. BIOC. ...,08.775.979/0002-62,RUA TABELIÃO IDELFONSO CAVALCANTI,455,NaN,CENTRO,62010-000,GASOLINA,01/01/2025,"6,29",NaN,R$ / litro,RAIZEN
1,NE,CE,SOBRAL,ECONOGÁS DO BRASIL DIST. DERIV. DE PET. BIOC. ...,08.775.979/0002-62,RUA TABELIÃO IDELFONSO CAVALCANTI,455,NaN,CENTRO,62010-000,GASOLINA ADITIVADA,01/01/2025,"6,49",NaN,R$ / litro,RAIZEN
2,NE,CE,SOBRAL,ECONOGÁS DO BRASIL DIST. DERIV. DE PET. BIOC. ...,08.775.979/0002-62,RUA TABELIÃO IDELFONSO CAVALCANTI,455,NaN,CENTRO,62010-000,DIESEL S10,01/01/2025,"6,19",NaN,R$ / litro,RAIZEN
3,NE,CE,SOBRAL,ECONOGÁS DO BRASIL DIST. DERIV. DE PET. BIOC. ...,08.775.979/0002-62,RUA TABELIÃO IDELFONSO CAVALCANTI,455,NaN,CENTRO,62010-000,ETANOL,01/01/2025,"5,19",NaN,R$ / litro,RAIZEN
4,NE,CE,SOBRAL,V.C.EMPREENDIMENTOS LTDA,03.551.935/0002-35,AVENIDA JOSE EUCLIDES FERREIRA GOMES,30,POSTO FLASH,CORACAO DE JESUS,62043-070,GASOLINA,01/01/2025,"6,53",NaN,R$ / litro,RAIZEN


,Regiao - Sigla,Estado - Sigla,Municipio,Revenda,CNPJ da Revenda,Nome da Rua,Numero Rua,Complemento,Bairro,Cep,Produto,Data da Coleta,Valor de Venda,Valor de Compra,Unidade de Medida,Bandeira
429518,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
429519,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
429520,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
429521,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
429522,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 429523 entries, 0 to 429522
Data columns (total 16 columns):
 #   Column             Non-Null Count   Dtype  
---  ------             --------------   -----  
 0   Regiao - Sigla     420409 non-null  object 
 1   Estado - Sigla     420409 non-null  object 
 2   Municipio          420409 non-null  object 
 3   Revenda            420409 non-null  object 
 4   CNPJ da Revenda    420409 non-null  object 
 5   Nome da Rua        420409 non-null  object 
 6   Numero Rua         420351 non-null  object 
 7   Complemento        96615 non-null   object 
 8   Bairro             419547 non-null  object 
 9   Cep                420409 non-null  object 
 10  Produto            420409 non-null  object 
 11  Data da Coleta     420409 non-null  object 
 12  Valor de Venda     420409 non-null  object 
 13  Valor de Compra    0 non-null       float64
 14  Unidade de Medida  420409 non-null  object 
 15  Bandeira           420409 non-null  object 
dtypes:

None

In [ ]:
# Contar a frequência de cada valor único
frequencia = df['Unidade de Medida'].value_counts()
print(frequencia)

### Análise Preliminar - Arquivos de Preços de Combustíveis (Amostra)
*Análise baseada em amostras dos arquivos de 2019-02 e 2025-01.*

#### Estrutura Geral
* **Volume:** São arquivos grandes, com centenas de milhares de linhas e 16 colunas.
* **Qualidade dos Dados:** Identificamos três problemas de qualidade importantes:
    1.  **`DtypeWarning` (Aviso de Tipos Mistos):** O aviso no arquivo de 2025 indica que o Pandas encontrou inconsistências nos tipos de dados ao ler o arquivo. **Ação:** Nosso processo de ETL na Fase 2, onde forçaremos a conversão para os tipos corretos, resolverá este aviso.
    2.  **Linhas Vazias no Final:** O arquivo de 2025 apresenta várias linhas completamente vazias (`NaN`) no final do arquivo. **Ação:** Precisaremos adicionar uma etapa no ETL para remover essas linhas vazias.
    3.  **Valores Nulos (Missing Values):** Há uma grande quantidade de valores nulos, especialmente nas colunas `Complemento` e, de forma crítica, na `Valor de Compra`.

#### Colunas de Dimensão (Contexto)
* **Localização (Muito Detalhada):** Este arquivo enriquece enormemente nossa visão de localização, com `Regiao - Sigla`, `Estado - Sigla`, `Municipio`, `Bairro`, `Cep`, `Nome da Rua`. Estas colunas serão a base para uma `d_localidade` muito robusta.
* **Revendedor (Nova Dimensão Confirmada):** As colunas `Revenda`, `CNPJ da Revenda` e `Bandeira` confirmam a necessidade de criarmos uma Tabela Dimensão `d_revendedor`.
* **Produto e Tempo:**
    * `Produto`: Confirma a conexão com nossa `d_produto`.
    * `Data da Coleta`: Lido como texto (**`object`**). **Ação Crítica:** Precisará ser convertida para um tipo de data (`datetime`) para permitir análises temporais. Será a chave para nossa `d_calendario`.

#### Colunas Fato (As Métricas)
* **`Valor de Venda`**: Lido como texto (**`object`**). **Ação Crítica:** Precisa ser convertido para um tipo numérico (`DECIMAL`), tratando a vírgula como separador decimal.
* **`Valor de Compra`**: Lido como `object` (2019) ou `float64` (2025). **Problema de Qualidade Grave:** Esta coluna é muito pouco confiável. Tem muitos valores faltando em 2019 e está **completamente vazia** em 2025. **Decisão de Modelagem:** Devemos considerar se vale a pena incluir esta métrica, ou se devemos focarno `Valor de Venda` que é mais consistente.
* **`Unidade de Medida`**: Lido como texto (**`object`**). É uma dimensão que dá contexto aos fatos. Será importante para garantir que estamos comparando as mesmas unidades (ex: `R$ / litro`).

#### Padronização
* Os nomes das colunas contêm hífens, espaços e acentos. Todos precisarão ser padronizados para o formato `snake_case`.